In [1]:
import pywavefront
import os
import numpy as np
import open3d as o3d
import bpy
import torch
from utils import pointPointDistance, Write3dcoord_toply, is_in_distance, cal_conn, getPathCoverage, cal_idx_from_coord, calculate_coverage, output_visualizable_faces, output_visualizable_faces_and_matrix
import random
#Method document:
#https://docs.google.com/document/d/1BG6a_NediVJCOH_1wTGCxYMteQseIQWRii4sys89h8k/edit?usp=sharing


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
WARN (bgl): source/blender/python/generic/bgl.c:2654 BPyInit_bgl: 'bgl' imported without an OpenGL backend. Please update your add-ons to use the 'gpu' module. In Blender 4.0 'bgl' will be removed.


Step1: Generate viewpoint space
input: distance threshold, geometric model, linear parameters(xmin,xmax,ymin,ymax,zmin,zmax,step)
output: pointcloud_out.ply - the available viewpoints
env_mat - a matrix in (n_viewpoints, 4). Column 0 indicates if the point is reachable. Colume 1-3 indicates the coordinates of the point.

In [18]:
def generate_viewpoints(dist_thres, lin_parameters, model_path, save_path):
    """
    Generate viewpoints aronud the object.
    Parameters:
        dist_thres - Float  The minimum distance between viewpoint and faces. The viewpoints with distance less than the threshold will be deleted.
        model_path - String  The file path of the model. The format should be obj
        lin_parameters:(x_extra,x_num,y_extra,y_num,z_extra,z_num) - (Float,Float,Float,Float,Float,Float)
        x_extra: the extra distance beyond the maximum/minimum x value considered for viewpoint space.
        x_num: the number of sampling values on x direction.
    Returns:
        (out_points_list,in_points_list) - (list,list) 
        out_points_list is the list of viewpoints outside the distance range. in_points_list is within the range, which should be excluded. 
        The output includes 2 files: in.ply, out.ply, which can be opened by modeling software for visualization of viewpoints.
    """
    xmin,xmax,ymin,ymax,zmin,zmax,step=lin_parameters
    #Read the obj file to vertices and faces
    scene = pywavefront.Wavefront(model_path, collect_faces=True)
    vertices=np.array(scene.vertices)

    #Read to faces
    faces_list=[]
    for mesh in scene.mesh_list:
        faces_list.append(mesh.faces)       

    #Find the range of vertices for the whole model
    scene_box = (scene.vertices[0], scene.vertices[0])   
    for vertex in scene.vertices:
        min_v = [min(scene_box[0][i], vertex[i]) for i in range(3)]
        max_v = [max(scene_box[1][i], vertex[i]) for i in range(3)]
        scene_box = (min_v, max_v)

    xspace=np.arange(xmin,xmax,step)
    yspace=np.arange(ymin,ymax,step) 
    zspace=np.arange(zmin,zmax,step)

    #Find the range of vertices for each part, the results are saved in part_box_list
    #part_box[0][0]: min_x part_box[1][0]: max_x part_box[0][1]: min_y part_box[1][1]: max_y  part_box[0][2]: min_z part_box[1][2]: max_z
    part_box_list=[]
    for faces in faces_list:
        vertices_in_part=[]
        for face in faces:
            vertices_in_part.append(vertices[face[0]])
            vertices_in_part.append(vertices[face[1]])
            vertices_in_part.append(vertices[face[2]])
        part_box = (vertices_in_part[0], vertices_in_part[0]) 
        for vertex in vertices_in_part:
            min_v=[min(part_box[0][i], vertex[i]) for i in range(3)]
            max_v=[max(part_box[1][i], vertex[i]) for i in range(3)]
            part_box=(min_v,max_v)
        part_box_list.append(part_box)
    #initialize out_points_list and in_points_list for saving the viewpoints
    out_points_list=[]
    in_points_list=[]
    all_points_list=[]
    #Loop for each viewpoint in the space    
    for x in xspace:
        for y in yspace:
            for z in zspace:
                #for each part of the model, calculate the distance between viewpoint and part box
                part_idx=0
                is_point_in=False
                for part_box in part_box_list:
                    #judge if each coordinate of the point lies in the range of the part.   
                    is_x_in=(x<part_box[1][0] and x>part_box[0][0])
                    is_y_in=(y<part_box[1][1] and y>part_box[0][1])
                    is_z_in=(z<part_box[1][2] and z>part_box[0][2])
                    #if the coordinate of the point is within the range, then the distance on this direction won't be considered.
                    #if not, then the maximum and minimum value for the part box will be used for calculating the distance respectively.
                    if is_x_in:
                        x_list=[x]
                    else:
                        x_list=[part_box[0][0],part_box[1][0]]
                    if is_y_in:
                        y_list=[y]
                    else:
                        y_list=[part_box[0][1],part_box[1][1]]                    
                    if is_z_in:
                        z_list=[z]
                    else:
                        z_list=[part_box[0][2],part_box[1][2]]
                    #calculate the minimum distance between the viewpoint and the part box
                    min_dist=pointPointDistance(np.array([x,y,z]),np.array([x_list[0],y_list[0],z_list[0]]))
                    for vertice_x in x_list:
                        for vertice_y in y_list:
                            for vertice_z in z_list:
                                dist=pointPointDistance(np.array([x,y,z]),np.array([vertice_x,vertice_y,vertice_z]))
                                min_dist=min(min_dist,dist)
                    point=np.array([x,y,z])
                    faces=faces_list[part_idx]
                    #if the minimum distance between the viewpoint and the part box is smaller than the threshold, 
                    #then calculate the distance between the viewpoint and each face in the part to decide if the viewpoint is too close to the faces
                    if min_dist<dist_thres:
                        if is_in_distance(point,faces,vertices,dist_thres):
                            is_point_in=True
                    part_idx+=1

                if is_point_in:
                    in_points_list.append(point)
                    #print(point,'in',min_dist)
                    all_points_list.append(np.hstack((0,point)))
                else:
                    out_points_list.append(point)
                    #print(point,'out',min_dist)
                    all_points_list.append(np.hstack((1,point)))
                print(point,end='\r')
    #output the results to ply file

    Write3dcoord_toply(np.array(out_points_list),save_path+'pointcloud_out.ply')
    Write3dcoord_toply(np.array(in_points_list),save_path+'pointcloud_in.ply')
    print('viewpoints saved')
    all_points_array=np.asarray(all_points_list)
    #all_points_array is an array with 4 columns. Column 0 indicates if the point is reachable. Colume 1-3 indicates the coordinates of the point.
    np.savetxt(save_path+'env_mat',all_points_array)
    print('env_mat saved')
    return (out_points_list,in_points_list)

In [2]:
xmin,xmax,ymin,ymax,zmin,zmax,step=-23,0,-13,8,1,11,1
lin_parameters=(xmin,xmax,ymin,ymax,zmin,zmax,step)
disp_thres=0.5
vis_dist=10
inclination_angle=45
inclination_radius=inclination_angle/180*np.pi
whole_model_path='3277_ground.obj'
bridge_model_path='3277_noground.obj'
ground_model_path='3277_groundonly.obj'
visible_model_path='visualizable.obj'

save_path='safedist='+str(disp_thres)+'_visdist='+str(vis_dist)+'_step='+str(step)+'/'

In [20]:
#step 1: calculate the viewpoint space

isExist = os.path.exists(save_path)
if not isExist:
    # Create a new directory because it does not exist
    os.makedirs(save_path)
#define optimization parameters
generate_viewpoints(disp_thres,lin_parameters, whole_model_path, save_path)

Unimplemented OBJ format statement 's' on line 's 0'


KeyboardInterrupt: 

step 2: calculate the viewpoint space
Input: env_mat
output: connection

In [6]:
env_coord_map=np.loadtxt(save_path+'env_mat')
conn=np.zeros((env_coord_map.shape[0],env_coord_map.shape[0]))#if there is obstacle between two viewpoints

point1_idx=0
for info1 in env_coord_map:
    #info1 is a NDArray(4) including the information of point 1
    point1=info1[1:4]
    point2_idx=0
    for info2 in env_coord_map:
        point2=info2[1:4]
        conn[point1_idx][point2_idx]=cal_conn(point1,point2,env_coord_map,lin_parameters)
        point2_idx+=1
    point1_idx+=1
    print('calculating connection for viewpoint: '+str(point1_idx)+'/'+str(env_coord_map.shape[0]),end='\r')
np.savetxt(save_path+'connection',conn,fmt="%d")
print('connection saved')

connection savedection for viewpoint: 4830/4830


step 3: calculate neighbor matrix indicating the indices of the points near a point

In [7]:
def generate_neighbor_matrix(lin_parameters, conn, env_coord_map):
    xmin,xmax,ymin,ymax,zmin,zmax,step=lin_parameters
    n_vp=env_coord_map.shape[0]
    neighbor_matrix=np.full((n_vp,27),-1)#Neighbor matrix records the information of all the neighbor viewpoints of a viewpoint. The shape of neighbor matrix is num_waypoints*27
    xspace=np.arange(xmin,xmax,step)
    yspace=np.arange(ymin,ymax,step) 
    zspace=np.arange(zmin,zmax,step)
    for x in xspace:
        for y in yspace:
            for z in zspace:
                point=np.array([x,y,z])
                idx=cal_idx_from_coord(point,lin_parameters)
                neighbor_counter=0
                for dx in (-step,0,step):
                    for dy in (-step,0,step):
                        for dz in (-step,0,step):
                            #check if the point is inside the range from three diretions 
                            is_in_range=(xmin<=x+dx<xmax) and (ymin<=y+dy<ymax) and (zmin<=z+dz<zmax)
                            is_valid=1 
                            if is_in_range:
                                neighbor_idx=cal_idx_from_coord(point+np.array([dx,dy,dz]),lin_parameters)                                
                                is_valid=env_coord_map[neighbor_idx,0]
                                is_connected=conn[idx,neighbor_idx]
                            if is_in_range and is_valid and is_connected:
                                neighbor_matrix[idx,neighbor_counter]=neighbor_idx
                            neighbor_counter+=1
    return neighbor_matrix


env_coord_map=np.loadtxt(save_path+'env_mat')
neighbor_matrix=generate_neighbor_matrix(lin_parameters,conn,env_coord_map)

conn=np.loadtxt(save_path+'connection')

np.savetxt(save_path+'neighbor_matrix',neighbor_matrix,fmt="%d")
print('neighbor_matrix saved')




neighbor_matrix saved


step 4: genetate visibility array for each viewpoint

Input: env_mat

output: connection

Generate visibility files for the valid bridge model

In [21]:
vp_path='pointcloud_out.ply'
pcd=o3d.io.read_point_cloud(save_path+vp_path)
view_points=np.asarray(pcd.points)
#view_points=view_points[0:10]
n_vp=view_points.shape[0]
rotations=np.zeros((n_vp,3))##just for placeholder. No longer effective
vis_path='vis_dist='+str(vis_dist)+'_angle='+str(inclination_angle)+'_noFOV/'

#load ground faces
ground_faces=np.zeros((3,0,3))
bpy.ops.object.select_all(action='SELECT')
bpy.ops.object.delete(use_global=False, confirm=False)
bpy.ops.import_scene.obj(filepath=ground_model_path)
bpy.ops.object.select_all(action='DESELECT')
for obj in bpy.data.objects:
    if obj.type == 'MESH':
        for face in obj.data.polygons:
            vertex_indices=face.vertices
            vertices = [obj.data.vertices[i].co for i in vertex_indices]
            vertices_np=np.asarray(vertices)
            ground_faces=np.concatenate((ground_faces, vertices_np[:,np.newaxis,:]), axis=1)
isExist = os.path.exists(vis_path)
if not isExist:
    # Create a new directory because it does not exist
    os.makedirs(vis_path)
vis_area=getPathCoverage(bridge_model_path,view_points,18,rotations,90/180*np.pi,False,vis_dist,inclination_radius,vis_path,ground_faces) 

(  0.0001 sec |   0.0001 sec) Importing OBJ '3277_groundonly.obj'...
  (  0.0003 sec |   0.0002 sec) Parsing OBJ file...
    (  0.0010 sec |   0.0007 sec) Done, loading materials and images...
    (  0.0013 sec |   0.0010 sec) Done, building geometries (verts:24 faces:44 materials: 1 smoothgroups:1) ...
    (  0.0025 sec |   0.0022 sec) Done.
  (  0.0026 sec |   0.0025 sec) Finished importing: '3277_groundonly.obj'
Progress: 100.00%

NVIDIA RTX A6000 1
NVIDIA RTX A6000 1
AMD Ryzen Threadripper PRO 3955WX 16-Cores 0
NVIDIA RTX A6000 1
NVIDIA RTX A6000 1
(  0.0000 sec |   0.0000 sec) Importing OBJ '3277_noground.obj'...
  (  0.0001 sec |   0.0001 sec) Parsing OBJ file...
    (  0.1214 sec |   0.1213 sec) Done, loading materials and images...
    (  0.1220 sec |   0.1219 sec) Done, building geometries (verts:5399 faces:10650 materials: 1 smoothgroups:1) ...
    (  0.2955 sec |   0.2954 sec) Done.
  (  0.2956 sec |   0.2956 sec) Finished importing: '3277_noground.obj'
Progress: 100.00%



 step 5:genetate visibility matrix and area array

In [24]:

bpy.context.scene.render.engine = 'CYCLES'
bpy.context.scene.cycles.device = 'GPU'
bpy.context.preferences.addons['cycles'].preferences.compute_device_type = 'CUDA'
bpy.context.preferences.addons['cycles'].preferences.get_devices()
for d in bpy.context.preferences.addons['cycles'].preferences.devices:
    if d["name"] == 'AMD Ryzen Threadripper PRO 3955WX 16-Cores':
        d["use"] = 0
    else:
        d["use"] = 1
    print(d["name"],d["use"])
bpy.ops.object.select_all(action='SELECT')
bpy.ops.object.delete(use_global=False, confirm=False)
bpy.ops.import_scene.obj(filepath=bridge_model_path)
bpy.ops.object.select_all(action='DESELECT')

#calculate the areas of faces
areas=[]
for obj in bpy.data.objects:
    if obj.type == 'MESH':
        for face in obj.data.polygons:
            areas.append(face.area)
areas=np.array(areas)
np.savetxt(save_path+'areas',areas)

#generate visibility matrix
env_coord_map=np.loadtxt(save_path+'env_mat')
n_vp=env_coord_map.shape[0] #number of viewpoints
vis_mat=np.zeros((areas.shape[0],0))
dir_list=[]
face_list=[]
txt_idx=0
vis_path='vis_dist='+str(vis_dist)+'_angle='+str(inclination_angle)+'_noFOV/'
vis_arr=np.loadtxt(vis_path+'vis0.txt')[:,1]
for idx_vp in range(n_vp):
    if env_coord_map[idx_vp,0]==1:
        vis_arr=np.loadtxt(vis_path+'vis'+str(txt_idx)+'.txt')[:,1]
        vis_arr=vis_arr.reshape((vis_arr.shape[0],1))
        dir_arr=np.loadtxt(vis_path+'direction'+str(txt_idx)+'.txt')
        face_arr=np.loadtxt(vis_path+'face'+str(txt_idx)+'.txt')
        txt_idx+=1
    else:
        face_arr=np.zeros(3,)
        vis_arr=np.zeros((vis_arr.shape[0],1))
        dir_arr=np.zeros((vis_arr.shape[0],3))
        
    vis_mat=np.hstack((vis_mat,vis_arr))
    dir_list.append(dir_arr)
    face_list.append(face_arr)

    print(txt_idx,idx_vp)
n_out = txt_idx
dir_mat=np.asarray(dir_list)

face_mat=np.asarray(face_list)
np.savetxt(save_path+'vis_matrix',vis_mat,fmt="%d")
np.save(save_path+'dir_matrix',dir_mat)
np.savetxt(save_path+'face_matrix',face_mat)

NVIDIA RTX A6000 1
NVIDIA RTX A6000 1
AMD Ryzen Threadripper PRO 3955WX 16-Cores 0
NVIDIA RTX A6000 1
NVIDIA RTX A6000 1
(  0.0000 sec |   0.0000 sec) Importing OBJ '3277_noground.obj'...
  (  0.0001 sec |   0.0001 sec) Parsing OBJ file...
    (  0.1232 sec |   0.1231 sec) Done, loading materials and images...
    (  0.1238 sec |   0.1237 sec) Done, building geometries (verts:5399 faces:10650 materials: 1 smoothgroups:1) ...
    (  0.2978 sec |   0.2977 sec) Done.
  (  0.2980 sec |   0.2979 sec) Finished importing: '3277_noground.obj'
Progress: 100.00%

0 0
0 1
0 2
0 3
1 4
2 5
3 6
4 7
5 8
6 9
6 10
6 11
6 12
6 13
7 14
8 15
9 16
10 17
11 18
12 19
12 20
12 21
12 22
12 23
13 24
14 25
15 26
16 27
17 28
18 29
18 30
18 31
18 32
18 33
19 34
20 35
21 36
22 37
23 38
24 39
24 40
24 41
24 42
24 43
25 44
26 45
27 46
28 47
29 48
30 49
30 50
30 51
30 52
30 53
31 54
32 55
33 56
34 57
35 58
36 59
36 60
36 61
36 62
36 63
37 64
38 65
39 66
40 67
41 68
42 69
42 70
42 71
42 72
42 73
42 74
42 75
43 76
44 77

step 6: calculate the proportion of all visible faces and output visible faces

In [5]:
vp_path='pointcloud_out.ply'
pcd=o3d.io.read_point_cloud(save_path+vp_path)
view_points=np.asarray(pcd.points)
sights=np.zeros_like(view_points)
sights[:,0]=1
vis_mat=np.loadtxt(save_path+'vis_matrix')
dir_mat=np.load(save_path+'dir_matrix.npy')
area_arr=np.loadtxt(save_path+'areas')
face_mat=np.loadtxt(save_path+'face_matrix')
VIS_MAT_tensor=torch.tensor(vis_mat, dtype=torch.float32)
DIR_MAT_tensor=torch.tensor(dir_mat, dtype=torch.float32)
AREA_ARR_tensor=torch.tensor(area_arr, dtype=torch.float32)
VIS_MAT_tensor = VIS_MAT_tensor.to('cuda')
DIR_MAT_tensor = DIR_MAT_tensor.to('cuda')
AREA_ARR_tensor = AREA_ARR_tensor.to('cuda')

view_points_tensor=torch.tensor(view_points, dtype=torch.float32)
sights_tensor=torch.tensor(sights, dtype=torch.float32)
view_points_tensor = view_points_tensor.to('cuda')
sights_tensor = sights_tensor.to('cuda')
coverage,vis_arr=calculate_coverage(view_points_tensor,sights_tensor,VIS_MAT_tensor,DIR_MAT_tensor,lin_parameters,AREA_ARR_tensor,-1)
vis_arr=vis_arr.to('cpu')
vis_arr=np.asarray(vis_arr)
model_range=np.array([(-22.666, -12.796), (-17.451, 7.346), (-0.675, 7.346),(-5.890, -12.796)]) #the faces out of this range will be cut off
idx_arr=output_visualizable_faces_and_matrix(bridge_model_path,vis_arr,save_path,model_range)
filtered_vis_mat=vis_mat[idx_arr][:]

filtered_dir_mat=dir_mat[:,idx_arr,:]
filtered_area_mat=area_arr[idx_arr]
print(coverage)
np.savetxt(save_path+'filtered_vis_matrix',filtered_vis_mat,fmt="%d")
np.save(save_path+'filtered_dir_matrix',filtered_dir_mat)
np.savetxt(save_path+'filtered_area_matrix',filtered_area_mat)

(  0.0001 sec |   0.0001 sec) Importing OBJ '3277_noground.obj'...
  (  0.0004 sec |   0.0003 sec) Parsing OBJ file...
    (  0.2088 sec |   0.2083 sec) Done, loading materials and images...
    (  0.2095 sec |   0.2091 sec) Done, building geometries (verts:5399 faces:10650 materials: 1 smoothgroups:1) ...
    (  0.3735 sec |   0.3730 sec) Done.
  (  0.3735 sec |   0.3734 sec) Finished importing: '3277_noground.obj'
Progress: 100.00%

    (  0.0019 sec |   0.0000 sec) OBJ Export path: 'safedist=0.5_visdist=10_step=1/visualizable.obj'
          (  0.0025 sec |   0.0004 sec) Finished writing geometry of 'Cube'.
          (  0.0026 sec |   0.0000 sec) Finished writing geometry of 'Light'.
          (  0.0027 sec |   0.0000 sec) Finished writing geometry of 'Camera'.
Error: Object does not have geometry data
Error: Object does not have geometry data
          (  0.0628 sec |   0.0601 sec) Finished writing geometry of '3277_nofence'.
      (  0.0629 sec |   0.0610 sec) Finished exporting ge

Output the visible faces from all the viewpoints

In [ ]:
import bpy
import numpy as np
import open3d as o3d
import bmesh

bpy.ops.import_scene.obj(filepath='bridge_mesh_whole.obj')


n_vp= n_out #set to the number of viewpoints with no collision
vis_globle=np.loadtxt('vis_dist=20_angle=45_noFOV/vis0.txt')
for idx_vp in range(n_vp):
    vis_local=np.loadtxt('vis_dist=20_angle=45_noFOV/vis'+str(idx_vp)+'.txt')

    vis_globle[:,1]+=vis_local[:,1]
    print(idx_vp)
np.savetxt('vis_dist=20_angle=45_noFOV/vis_all.txt',vis_globle)
for idx_f in range(vis_globle.shape[0]):
    if vis_globle[idx_f,1]>0:
        vis_globle[idx_f,1]=1
cc=0
for obj in bpy.data.objects:
    if obj.type == 'MESH':
        for face in obj.data.polygons:
            face.select=False
areas=[]
for obj in bpy.data.objects:
    if obj.type == 'MESH':
        for face in obj.data.polygons:
            areas.append(face.area)

for obj in bpy.data.objects:

    if obj.type == 'MESH'and obj.name!='Cube':
        bpy.ops.object.mode_set(mode='EDIT')
        bm=bmesh.from_edit_mesh(obj.data)
        bm.faces.ensure_lookup_table()
        for face in bm.faces:
            
            if vis_globle[cc][1]==1:
                face.select=True

            else:
                bm.faces.ensure_lookup_table()
                bmesh.ops.delete(bm,geom=[face],context='FACES')                
            cc+=1


# Delete the default cube object

bpy.ops.export_scene.obj(filepath='test.obj')